In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import glob
from datetime import datetime, timedelta

In [2]:
files = glob.glob('*.csv')

df_list = []

for file in files:
    data = pd.read_csv(file, lineterminator='\n')
    df_list.append(data)
    
df = pd.concat(df_list)

<ipython-input-2-0721a045282f>:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat(df_list)


In [3]:
df.head()

,Unnamed: 0,brand,model,rating,review,review_date,review_location,reviewer_name,size,trial,trial_nights,warranty,warranty_years
0,0,purple,The Purple Mattress,5.0,I am a hard worker with way more to physically...,02/22/2021,United States,"<div class=""author"">Jessica G.</div>",NaN,100.0,NaN,10-Year,NaN
1,1,purple,The Purple Mattress,5.0,First off I am a disabled veteran with insomni...,09/19/2017,"Warrenton, Virginia","<div class=""author"">Jared B.</div>",NaN,100.0,NaN,10-Year,NaN
2,2,purple,The Purple Mattress,1.0,This mattress is well made and the process of ...,03/05/2021,United States,"<div class=""author"">LeAnn M.</div>",NaN,100.0,NaN,10-Year,NaN
3,3,purple,The Purple Mattress,3.0,The regular twin foam mattress was marked as a...,03/04/2021,United States,"<div class=""author"">Kathryn W.</div>",NaN,100.0,NaN,10-Year,NaN
4,4,purple,The Purple Mattress,4.0,We had a premium mattress for years (Sterns an...,03/04/2021,United States,"<div class=""author"">Adam R.</div>",NaN,100.0,NaN,10-Year,NaN


In [4]:
df.drop(['Unnamed: 0', 'size', 'trial_nights', 'warranty_years'], axis=1, inplace=True)

In [5]:
df.reset_index(drop=True, inplace=True)

In [6]:
df.head()

,brand,model,rating,review,review_date,review_location,reviewer_name,trial,warranty
0,purple,The Purple Mattress,5.0,I am a hard worker with way more to physically...,02/22/2021,United States,"<div class=""author"">Jessica G.</div>",100.0,10-Year
1,purple,The Purple Mattress,5.0,First off I am a disabled veteran with insomni...,09/19/2017,"Warrenton, Virginia","<div class=""author"">Jared B.</div>",100.0,10-Year
2,purple,The Purple Mattress,1.0,This mattress is well made and the process of ...,03/05/2021,United States,"<div class=""author"">LeAnn M.</div>",100.0,10-Year
3,purple,The Purple Mattress,3.0,The regular twin foam mattress was marked as a...,03/04/2021,United States,"<div class=""author"">Kathryn W.</div>",100.0,10-Year
4,purple,The Purple Mattress,4.0,We had a premium mattress for years (Sterns an...,03/04/2021,United States,"<div class=""author"">Adam R.</div>",100.0,10-Year


In [7]:
df.isna().sum()

brand                  0
model                  0
rating                 0
review                 0
review_date         2100
review_location     1744
reviewer_name        902
trial              22997
warranty           22997
dtype: int64

In [8]:
df.rename(columns={'trial': 'trial_nights',
                    'warranty': 'warranty_years'}, inplace=True)
df['warranty_years'] = df['warranty_years'].apply(lambda x: int(x.split('-')[0]) if isinstance(x, str) else x)

In [9]:
df[df['trial_nights'].isna()]

,brand,model,rating,review,review_date,review_location,reviewer_name,trial_nights,warranty_years
22997,purple,The Purple Mattress,5.0,I am a hard worker with way more to physically...,2021-02-22,United States,Jessica G.,NaN,NaN
22998,purple,The Purple Mattress,5.0,First off I am a disabled veteran with insomni...,2017-09-19,"Warrenton, Virginia",Jared B.,NaN,NaN
22999,purple,The Purple Mattress,1.0,This mattress is well made and the process of ...,2021-03-05,United States,LeAnn M.,NaN,NaN
23000,purple,The Purple Mattress,3.0,The regular twin foam mattress was marked as a...,2021-03-04,United States,Kathryn W.,NaN,NaN
23001,purple,The Purple Mattress,4.0,We had a premium mattress for years (Sterns an...,2021-03-04,United States,Adam R.,NaN,NaN
...,...,...,...,...,...,...,...,...,...
45989,Casper,Element Mattress,5.0,My granddaughter finds her new twin very comfo...,2018-07-24,"Clermont, FL",Tricia,NaN,NaN
45990,Casper,Element Mattress,5.0,"After years and years of sleeping on coils, my...",2018-07-24,"Los Angeles, CA",Jared,NaN,NaN
45991,Casper,Element Mattress,5.0,I have had a hard time sleeping since I broke ...,2018-07-24,"LARAMIE, WY",Megan,NaN,NaN
45992,Casper,Element Mattress,4.0,Firmer than I expected but best sleep ever. I ...,2018-07-24,"Robinson, IL",Joshua,NaN,NaN


In [10]:
df.dropna(subset=['trial_nights', 'warranty_years'], inplace=True)

In [11]:
df.loc[df['brand'] == 'purple', 
       'reviewer_name'] = df.loc[df['brand'] == 'purple', 
                                 'reviewer_name'].apply(lambda x: x.split('>')[1].split('<')[0])

In [12]:
today = datetime.today()

df.loc[df.brand == 'leesa', 'review_date'] = df.loc[df.brand == 'leesa', 'review_date'].apply(lambda x: \
                                    (today - timedelta(days=int(x))).strftime('%m-%d-%y') if len(x) <=2 else x)

In [13]:
for brand in df['brand'].unique():
    print(brand)
    print(df.loc[df['brand'] == brand, 'review_date'].head())
    print('----------\n')

purple
0    02/22/2021
1    09/19/2017
2    03/05/2021
3    03/04/2021
4    03/04/2021
Name: review_date, dtype: object
----------

Tempur Pedic
6000    Dec 16, 2020
6001     Dec 6, 2020
6002    Nov 28, 2020
6003    Nov 22, 2020
6004    Nov 12, 2020
Name: review_date, dtype: object
----------

serta
11600    April 20, 2020
11601    April 23, 2020
11602    April 25, 2020
11603    April 26, 2020
11604    April 27, 2020
Name: review_date, dtype: object
----------

leesa
12331    03-05-21
12332    02-19-21
12333    02-17-21
12334    03-05-21
12335    02-19-21
Name: review_date, dtype: object
----------

Sealy
15799    Jan 25, 2021
15800    Jan 25, 2021
15801    Jan 25, 2021
15802    Jan 25, 2021
15803    Jan 25, 2021
Name: review_date, dtype: object
----------

Casper
17899    Jan 3, 2021
17900    Jan 3, 2021
17901    Jan 3, 2021
17902    Jan 3, 2021
17903    Jan 3, 2021
Name: review_date, dtype: object
----------



In [14]:
purp_dates = df.loc[df['brand'] == 'purple', 'review_date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))
leesa_dates = df[df['brand'] == 'leesa']['review_date'].apply(lambda x: datetime.strptime(x, '%m-%d-%y'))
serta_dates = df[df['brand'] == 'serta']['review_date'].apply(lambda x: datetime.strptime(x, '%B %d, %Y'))
casper_dates = df[df['brand']  == 'Casper']['review_date'].apply(lambda x: datetime.strptime(x, '%b %d, %Y'))
sealy_dates = df[df['brand']  == 'Sealy']['review_date'].apply(lambda x: datetime.strptime(x, '%b %d, %Y'))
tp_dates = df[df['brand']  == 'Tempur Pedic']['review_date'].apply(lambda x: datetime.strptime(x, '%b %d, %Y'))

In [15]:
df['review_date'] = datetime.now()

In [16]:
df.loc[df['brand'] == 'purple','review_date'] = purp_dates
df.loc[df['brand'] == 'leesa', 'review_date'] = leesa_dates
df.loc[df['brand'] == 'serta', 'review_date'] = serta_dates
df.loc[df['brand'] == 'Sealy', 'review_date'] = sealy_dates
df.loc[df['brand'] == 'Casper', 'review_date'] = casper_dates
df.loc[df['brand'] == 'Tempur Pedic', 'review_date'] = tp_dates

df

,brand,model,rating,review,review_date,review_location,reviewer_name,trial_nights,warranty_years
0,purple,The Purple Mattress,5.0,I am a hard worker with way more to physically...,2021-02-22,United States,Jessica G.,100.0,10.0
1,purple,The Purple Mattress,5.0,First off I am a disabled veteran with insomni...,2017-09-19,"Warrenton, Virginia",Jared B.,100.0,10.0
2,purple,The Purple Mattress,1.0,This mattress is well made and the process of ...,2021-03-05,United States,LeAnn M.,100.0,10.0
3,purple,The Purple Mattress,3.0,The regular twin foam mattress was marked as a...,2021-03-04,United States,Kathryn W.,100.0,10.0
4,purple,The Purple Mattress,4.0,We had a premium mattress for years (Sterns an...,2021-03-04,United States,Adam R.,100.0,10.0
...,...,...,...,...,...,...,...,...,...
22992,Casper,Element Mattress,5.0,My granddaughter finds her new twin very comfo...,2018-07-24,"Clermont, FL",Tricia,100.0,10.0
22993,Casper,Element Mattress,5.0,"After years and years of sleeping on coils, my...",2018-07-24,"Los Angeles, CA",Jared,100.0,10.0
22994,Casper,Element Mattress,5.0,I have had a hard time sleeping since I broke ...,2018-07-24,"LARAMIE, WY",Megan,100.0,10.0
22995,Casper,Element Mattress,4.0,Firmer than I expected but best sleep ever. I ...,2018-07-24,"Robinson, IL",Joshua,100.0,10.0


In [17]:
df['review'] = df['review'].apply(lambda x: x.replace('\n', ' '))

In [18]:
df.to_csv('Final_cleaned_data_2.csv')